In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Hashing, Embedding, Flatten
import tensorflow as tf

Вспомогательные функции.

In [ ]:
def del_dublicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            result.append(item)
            seen.add(item)

    return result

Функции для нормализации данных.

In [ ]:
def code_hashes(data, output_dim, num_bins, hash_col_name, batch_size=128):
    """
    Кодирует хэши в ембеддинги.

    Неодходимые импорты:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Input, Hashing, Embedding, Flatten
    import tensorflow as tf
    import pandas as pd

    :param data: Pandas таблица (pd.DataFrame), в котором будут кодироваться хэши одного из столбцов.
    :param output_dim: Размерность эмбеддингов.
    :param num_bins: Количество корзин для раскладывания в них хэшей. Лучше поставить 2 * количество уникальных хэшей.
    :param hash_col_name: Имя столбца, в котором будут кодироваться хэши.
    :param batch_size: Размер батча, который будет использоваться для кодирования хэшей в keras модели.
    :return: data с кодированным столбцом, словарь хэш: эмбеддниг, keras модель для кодирования хэшей.
    """

    data = data.copy()  # копирование таблицы
    data[hash_col_name] = data[hash_col_name].astype(str)  # изменение типа данных для модели

    # создание модели для кодирования хэшей
    hash_encoder = Sequential()
    hash_encoder.add(Input((1,), dtype=tf.string))
    hash_encoder.add(Hashing(num_bins=num_bins))
    hash_encoder.add(Embedding(input_dim=num_bins, output_dim=output_dim))
    hash_encoder.add(Flatten())
    hash_encoder.build()

    # создание словаря с кодированными хэшами
    hashes = data[hash_col_name].unique()
    coded_hashes = hash_encoder.predict(hashes, batch_size=batch_size)
    coded_hashes_dict = dict(zip(hashes, coded_hashes))

    # создание таблицы с кодированными хэшами
    coded_data = [coded_hashes_dict[hash_] for hash_ in data[hash_col_name]]
    col_names = [hash_col_name + f'_embedding_{n}' for n in range(1, output_dim + 1)]
    coded_hashes_df = pd.DataFrame(coded_data, columns=col_names, index=data.index)

    # добавление кодированных хэшей к обучающим данным
    data = pd.concat([data, coded_hashes_df], axis=1)

    return data, coded_hashes_dict, hash_encoder

In [ ]:
def augment_date(all_data, date_column_name, categorical_columns=None):
    """
    На вход надо передавать объединённые признаки в обучающих данных и признаки для сабмита.

    Необходимых импортов нет.
    """
    
    if categorical_columns is None:
        categorical_columns = []

    all_data = all_data.copy()

    if pd.api.types.is_datetime64_any_dtype(all_data[date_column_name].dtype) and all_data[date_column_name].dt.tz is not None:
        all_data[date_column_name] = all_data[date_column_name].dt.tz_localize(None)

    min_date = all_data[date_column_name].min()
    all_data[date_column_name + '_year'] = all_data[date_column_name].dt.year
    all_data[date_column_name + '_quarter'] = all_data[date_column_name].dt.quarter
    all_data[date_column_name + '_month'] = all_data[date_column_name].dt.month
    all_data[date_column_name + '_day'] = all_data[date_column_name].dt.day
    # all_data[date_column_name + '_hour'] = all_data[date_column_name].dt.hour
    # all_data[date_column_name + '_minute'] = all_data[date_column_name].dt.minute
    all_data[date_column_name + '_month_since_start'] = (all_data[date_column_name].dt.year * 12 + all_data[date_column_name].dt.month
                                                        ) - (min_date.year * 12 + min_date.month)
    all_data[date_column_name + '_quarter_since_start'] = (all_data[date_column_name].dt.year * 4 + all_data[date_column_name].dt.quarter
                                                          ) - (min_date.year * 4 + min_date.quarter)
    all_data[date_column_name + '_years_since_start'] = all_data[date_column_name].dt.year - min_date.year
    all_data[date_column_name + '_days_since_start'] = (all_data[date_column_name] - min_date).dt.days
    all_data[date_column_name + '_is_weekend'] = (all_data[date_column_name].dt.weekday >= 5).astype(int).astype('category')
    updated_categorical_columns = categorical_columns + [date_column_name + '_is_weekend']
    all_data[date_column_name + '_weekday'] = all_data[date_column_name].dt.weekday.astype('category')
    updated_categorical_columns = updated_categorical_columns + [date_column_name + '_weekday']
    
    all_data.drop(date_column_name, axis=1, inplace=True)  # удаление этого столбца, так как он больше не нужен

    return all_data, del_dublicates(updated_categorical_columns)

In [ ]:
def apply_LabelEncoder(all_data, categorical_columns):
    """
    На вход надо передавать объединённые признаки в обучающих данных и признаки для сабмита.

    Необходимые импорты:
    from sklearn.preprocessing import LabelEncoder
    """

    all_data = all_data.copy()

    for cat_col in categorical_columns:
        le = LabelEncoder()
        all_data[cat_col] = le.fit_transform(all_data[cat_col])
        all_data[cat_col] = all_data[cat_col].astype('category')

    return all_data

In [ ]:
def apply_OHE(all_data, categorical_columns):
    """
    Применяет One-Hot Encoding к указанным категориальным колонкам. NaN значения тоже кодируются,
    как отдельная категория.

    На вход надо передавать объединённые признаки в обучающих данных и признаки для сабмита,
    чтобы гарантировать одинаковый набор колонок после кодирования.

    Необходимые импорты:
    import pandas as pd
    """

    all_data = all_data.copy()
    all_data = pd.get_dummies(all_data, columns=categorical_columns, dummy_na=False)

    return all_data

In [ ]:
def augment_isna(data, categorical_columns=None):
    """
    Добавляет в pd.DataFrame столбцы, которые говорят о наличии пропусков.

    Необходимые импорты:
    import pandas as pd
    """

    if categorical_columns is None:
        categorical_columns = []

    data = data.copy()

    nan_col_names = [col for col in data.columns if data[col].isna().any()]

    if not nan_col_names:
        return data, categorical_columns

    isna_df = data[nan_col_names].isna().astype(int)

    new_column_names_map = {col: f"{col}_isna" for col in nan_col_names}
    isna_df = isna_df.rename(columns=new_column_names_map)

    for col in isna_df.columns:
        isna_df[col] = isna_df[col].astype('category')

    data = pd.concat([data, isna_df], axis=1)

    updated_categorical_columns = categorical_columns + list(isna_df.columns)

    return data, del_dublicates(updated_categorical_columns)

In [ ]:
def code_text_with_char_tfidf_svd(data, text_col_name, n_components=20, 
                                  ngram_range=(3, 5), tfidf_max_features=20000, 
                                  random_state=42):
    """
    Кодирует многоязычные текстовые данные с помощью символьных n-грамм TF-IDF и SVD.
    Этот метод языконезависим и хорошо подходит как быстрый baseline.

    Надо вот так конкатенировать обучающие и тестовые данные (`ignore_index=True` !!!):
    all_data = pd.concat([train_data, submission_data], axis=0, ignore_index=True)

    Необходимые импорты:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD
    import pandas as pd

    :param data: pd.DataFrame с текстовыми столбцами.
    :param text_col_name: Имя столбца (str) или список имен столбцов (list) с текстом.
    :param n_components: Количество SVD компонент. Это итоговая размерность признаков.
    :param ngram_range: Кортеж (min_n, max_n). Задает диапазон длин n-грамм символов.
                        Например, (3, 5) будет использовать 3-, 4- и 5-граммы.
    :param tfidf_max_features: Максимальное количество самых частых n-грамм, которые
                               будут включены в словарь. Ограничивает размер матрицы.
    :param random_state: Random state для SVD для воспроизводимости результатов.
    :return: pd.DataFrame только с новыми сгенерированными признаками.
    """

    data_copy = data.copy()
    temp_col_name = None
    
    if isinstance(text_col_name, list):
        output_prefix = '_'.join(text_col_name)
        temp_col_name = f'__temp_combined_{output_prefix}'
        data_copy[temp_col_name] = data_copy[text_col_name].fillna('').agg(' '.join, axis=1)
        processing_col = temp_col_name
    elif isinstance(text_col_name, str):
        output_prefix = text_col_name
        processing_col = text_col_name
    else:
        raise TypeError("Параметр 'text_col_name' должен быть строкой или списком строк.")

    unique_texts = data_copy[[processing_col]].drop_duplicates().dropna()
    
    if unique_texts.empty:
        return pd.DataFrame(index=data.index)

    tfidf_vectorizer = TfidfVectorizer(
        analyzer='char_wb',
        ngram_range=ngram_range,
        max_features=tfidf_max_features
    )
    tfidf_matrix = tfidf_vectorizer.fit_transform(unique_texts[processing_col])

    svd = TruncatedSVD(n_components=n_components, random_state=random_state)
    svd_features = svd.fit_transform(tfidf_matrix)

    col_names = [f'{output_prefix}_char_tfidf_svd_{i}' for i in range(n_components)]
    svd_df = pd.DataFrame(svd_features, index=unique_texts.index, columns=col_names)
    
    data_with_features = data_copy.merge(svd_df, left_index=True, right_index=True, how='left')

    return data_with_features.drop(text_col_name, axis=1)

In [1]:
def code_text_with_transformers(data, text_col_name, no_components=None,
                                model_name='paraphrase-multilingual-MiniLM-L12-v2', 
                                batch_size=32, show_progress_bar=True):
    """
    Кодирует многоязычные текстовые данные в семантические эмбеддинги с помощью
    предобученной трансформерной модели и опционально сжимает их.

    Необходимые импорты:
    from sentence_transformers import SentenceTransformer
    import pandas as pd
    from sklearn.decomposition import PCA

    :param data: pd.DataFrame с текстовыми столбцами.
    :param text_col_name: Имя столбца (str) или список имен столбцов (list) с текстом.
    :param no_components: (Опционально) Целевое количество признаков. Если задано,
                          эмбеддинги будут сжаты с помощью PCA до этого размера.
    :param model_name: Название предобученной многоязычной модели из библиотеки 
                       sentence-transformers. 
                       'paraphrase-multilingual-MiniLM-L12-v2' - хороший баланс скорости и качества.
                       'paraphrase-multilingual-mpnet-base-v2' - медленнее, но качество выше.
    :param batch_size: Размер батча для кодирования (влияет на использование VRAM/RAM).
    :param show_progress_bar: Показывать ли прогресс-бар при кодировании.
    :return: pd.DataFrame со всеми исходными столбцами и новыми сгенерированными признаками.
    """
    data_copy = data.copy()
    temp_col_name = None
    
    # если на вход подан список колонок, объединяем их в одну временную
    if isinstance(text_col_name, list):
        output_prefix = '_'.join(text_col_name)
        temp_col_name = f'__temp_combined_{output_prefix}'
        data_copy[temp_col_name] = data_copy[text_col_name].fillna('').agg(' '.join, axis=1)
        processing_col = temp_col_name
    elif isinstance(text_col_name, str):
        output_prefix = text_col_name
        processing_col = text_col_name
    else:
        raise TypeError("параметр 'text_col_name' должен быть строкой или списком строк.")

    # работаем только с уникальными текстами для экономии ресурсов
    unique_texts = data_copy[[processing_col]].drop_duplicates().dropna()
    
    if unique_texts.empty:
        # если нет текстов для кодирования, добавляем пустые столбцы и возвращаем копию
        final_dim = no_components if no_components is not None else 384 # 384 - размерность MiniLM
        for i in range(final_dim):
            data_copy[f'{output_prefix}_embed_{i}'] = None
        return data_copy

    model = SentenceTransformer(model_name)

    # кодируем тексты в эмбеддинги
    embeddings = model.encode(
        unique_texts[processing_col].tolist(),
        batch_size=batch_size, 
        show_progress_bar=show_progress_bar,
        convert_to_numpy=True
    )

    # если указан no_components, сжимаем эмбеддинги с помощью pca
    if no_components is not None and isinstance(no_components, int) and no_components > 0:
        if no_components < embeddings.shape[1]:
            print(f"сжатие эмбеддингов с {embeddings.shape[1]} до {no_components} с помощью pca...")
            pca = PCA(n_components=no_components, random_state=42)
            embeddings = pca.fit_transform(embeddings)
        else:
            print(f"предупреждение: no_components ({no_components}) >= исходной размерности ({embeddings.shape[1]}). сжатие не выполняется.")

    # создаем датафрейм с эмбеддингами, готовый к объединению
    col_names = [f'{output_prefix}_embed_{i}' for i in range(embeddings.shape[1])]
    embedding_df = pd.DataFrame(embeddings, columns=col_names)
    embedding_df[processing_col] = unique_texts[processing_col].values
    
    # присоединяем эмбеддинги к копии исходных данных по текстовому полю
    data_with_features = data_copy.merge(embedding_df, on=processing_col, how='left')
    
    # удаляем временную колонку, если она создавалась
    if temp_col_name:
        data_with_features.drop(columns=[temp_col_name], inplace=True)
    
    # возвращаем всю таблицу с новыми признаками
    return data_with_features

In [ ]:
def preprocess_catnum_cols(all_data, cat_cols, num_cols, cat_num_cols):
    """
    Обрабатывает DataFrame, кодируя категориальные признаки и создавая 
    числовые копии для указанных столбцов. cat_cols, num_cols и cat_num_cols
    не должны пересекаться. Функцию следует использовать после заполнения NaN
    значений.

    Необходимые импорты:
    from sklearn.preprocessing import LabelEncoder

    :param all_data: pd.DataFrame, который является объединением обучающих данных и данных для сабмита.
    :param cat_cols: список категориальных столбцов.
    :param num_cols: список численных столбцов.
    :param cat_num_cols: список столбцов, которые стоит добавить в данные как категориальные, так и численные.
                         Изначально столбцы в cat_num_cols должны являться численными.

    :return: обновлённый all_data и новые cat_cols и num_cols.
    """

    # создание копий
    data = all_data.copy()
    final_cat_cols = cat_cols.copy()
    final_num_cols = num_cols.copy()

    le = LabelEncoder()  # создание кодировщника

    # кодирование столбцов, которые должны быть и категориальными, и численными
    for col in cat_num_cols:
        data[col] = data[col].astype('float32')  # изменение типа данных на числовой, чтобы он точно не был категориальным
        new_cat_col_name = f'{col}_cat'  # имя для кодированной копии столбца

        # создание кодированной копии столбца
        data[new_cat_col_name] = le.fit_transform(data[col].astype(str))
        data[new_cat_col_name] = data[new_cat_col_name].astype('category')

        final_num_cols.append(col)  # добавляем исходный столбец в список числовых столбцов
        final_cat_cols.append(new_cat_col_name)  # добавляем новый столбец в список категориальных столбцов

    # кодирование <<чисто>> категориальных столбцов
    for col in cat_cols:
        data[col] = le.fit_transform(data[col].astype(str))
        data[col] = data[col].astype('category')

    return data, final_cat_cols, final_num_cols

In [ ]:
def reduce_mem_usage(df):
    """
    проходит по всем столбцам датафрейма и изменяет тип данных
    числовых столбцов на минимально возможный для уменьшения использования памяти.
    
    :param df: pd.DataFrame для оптимизации.
    :return: pd.DataFrame с оптимизированными типами данных.
    """

    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Использование памяти до: {start_mem:.2f} mb')
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in str(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # для категориальных можно дополнительно использовать df[col].astype('category')
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Использование памяти после: {end_mem:.2f} mb')
    print(f'Сжато на {100 * (start_mem - end_mem) / start_mem:.1f}%')
    
    return df